##Plotting Utilities

In [ ]:
def plot_series(x, y, format="-", start=0, end=None, 
                title=None, xlabel=None, ylabel=None, legend=None ):
    """
    Visualizes time series data

    Args:
      x (array of int) - contains values for the x-axis
      y (array of int or tuple of arrays) - contains the values for the y-axis
      format (string) - line style when plotting the graph
      label (string) - tag for the line
      start (int) - first time step to plot
      end (int) - last time step to plot
      title (string) - title of the plot
      xlabel (string) - label for the x-axis
      ylabel (string) - label for the y-axis
      legend (list of strings) - legend for the plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))
    
    # Check if there are more than two series to plot
    if type(y) is tuple:

      # Loop over the y elements
      for y_curr in y:

        # Plot the x and current y values
        plt.plot(x[start:end], y_curr[start:end], format)

    else:
      # Plot the x and y values
      plt.plot(x[start:end], y[start:end], format)

    # Label the x-axis
    plt.xlabel(xlabel)

    # Label the y-axis
    plt.ylabel(ylabel)

    # Set the legend
    if legend:
      plt.legend(legend)

    # Set the title
    plt.title(title)

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()

##Callbacks

Make sure to include `callbacks` when performing model.fit():

`model.fit(x_train, y_train, epochs = 10, callbacks=[callbacks])`

In [ ]:
#creating a callback for computer vision

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training after reaching {accuracy_value} percent accuracy

    Args:
      epoch (integer) - index of epoch (required but not used in function def. below)
      logs (dict) - metric results from the training epoch
    '''

    #check accuracy 
    accuracy_value = 0.89
    if(logs.get('accuracy') > accuracy_value):

      #stop if threshold is met
      print(f'\nAccuracy is more than {accuracy_value} so cancelling training!')
      self.model.stop_training = True

#instantiate class
callbacks = myCallback()

##Synthetic Data Generation

In [ ]:
def trend(time, slope=0):
    """
    Generates synthetic data that follows a straight line given a slope value.

    Args:
      time (array of int) - contains the time steps
      slope (float) - determines the direction and steepness of the line

    Returns:
      series (array of float) - measurements that follow a straight line
    """

    # Compute the linear series given the slope
    series = slope * time

    return series

In [ ]:
def seasonal_pattern(season_time):
    """
    Just an arbitrary pattern, you can change it if you wish
    
    Args:
      season_time (array of float) - contains the measurements per time step

    Returns:
      data_pattern (array of float) -  contains revised measurement values according 
                                  to the defined pattern
    """

    # Generate the values using an arbitrary pattern
    data_pattern = np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))
    
    return data_pattern

In [ ]:
def seasonality(time, period, amplitude=1, phase=0):
    """
    Repeats the same pattern at each period

    Args:
      time (array of int) - contains the time steps
      period (int) - number of time steps before the pattern repeats
      amplitude (int) - peak measured value in a period
      phase (int) - number of time steps to shift the measured values

    Returns:
      data_pattern (array of float) - seasonal data scaled by the defined amplitude
    """
    
    # Define the measured values per period
    season_time = ((time + phase) % period) / period

    # Generates the seasonal data scaled by the defined amplitude
    data_pattern = amplitude * seasonal_pattern(season_time)

    return data_pattern


In [ ]:
def noise(time, noise_level=1, seed=None):
    """Generates a normally distributed noisy signal

    Args:
      time (array of int) - contains the time steps
      noise_level (float) - scaling factor for the generated signal
      seed (int) - number generator seed for repeatability

    Returns:
      noise (array of float) - the noisy signal
    """

    # Initialize the random number generator
    rnd = np.random.RandomState(seed)

    # Generate a random number for each time step and scale by the noise level
    noise = rnd.randn(len(time)) * noise_level
    
    return noise

##Preparing Features and Labels for Time Series Data

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  """
  Generates dataset windows 

  Args:
    series (array of float) - contains the values of the time series 
    window_size (int) - the number of time steps to include in the feature 
    batch_size (int) - the size of the batches that'll be fed into the model 
    shuffle_buffer (int) - buffer size to use for the shuffle method 

  Returns:
    dataset (TF Dataset) - TF Dataset containing time windows 
  """

  #Generate a TF Dataset from the series values 
  dataset = tf.data.Dataset.from_tensor_slices(series)

  #Window the data but only take those with the specified size 
  dataset = dataset.window(window_size + 1, shift = 1, drop_remainder=True)

  #Flatten the windows by putting its elements in a single batch
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

  #Create tuples with features and labels
  dataset = dataset.map(lambda window: (window[: -1], window[-1]))

  #Create batches of windows 
  dataset = dataset.batch(batch_size).prefetch(1)

  return dataset

##Model Prediction For Time Series Data


In [ ]:
def model_forecast(model, series, window_size, batch_size):
  """
  Uses an unput model to generate predictions on data windows 

  Args:
    model (TF Keras Model) - model that accepts data windows 
    series (array of float) - contains the values of the time series 
    window_size (int) - the number of time steps to include in the window 
    batch_size (int) - the batch size

  Returns:
    forecast (numpy array) - array containing predictions
  """

  #Generat a TF Dataset from the series values 
  dataset = tf.data.Dataset.from_tensor_slices(series)

  #Window the data but only take those with the specified size 
  dataset = dataset.window(window_size, shift=1, drop_remainder=True)

  #Flatten the windows by putting its elements in a single batch 
  dataset = dataset.flat_map(lambda w: w.batch(window_size))

  #Create batches of windows 
  dataset = dataset.batch(batch_size).prefetch(1)

  #Get predictions on the entire dataset
  forecast = model.predict(dataset)

  return forecast